In [ ]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')
wells_list = np.unique(las_data['well_name'].values).tolist()
len(wells_list)

In [ ]:
# загружаем wc
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
water_content_data.head()
las_data = pd.merge(las_data, water_content_data, on=main.keys_dict[main.kid_well])
print(len(np.unique(las_data['well_name'].values).tolist()))
las_data.head()

In [ ]:
header_data = pd.read_csv(main.data_dir + "\\coord.csv", delimiter=';')
header_data.head()
header_data = header_data.rename(columns={'num': 'well_name'})
header_data = header_data.rename(columns={'well': 'UWI'})
header_data.head()
del header_data['field']
del header_data['plastorig']

In [ ]:
las_data = pd.merge(las_data, header_data, on='UWI')
print(len(np.unique(las_data['well_name'].values).tolist()))
las_data.head()

In [ ]:
coord_data = las_data[['well_name', 'X-Coord', 'Y-Coord']]
coord_data = coord_data.drop_duplicates()
# coord_data = coord_data.reset_index()
print(coord_data.shape)
coord_data.head()

In [ ]:
coord_data['X-Coord'] = coord_data['X-Coord'].astype('int64')
coord_data['Y-Coord'] = coord_data['Y-Coord'].astype('int64')
coord_data.info()

In [ ]:
x_arr = coord_data['X-Coord']
y_arr = coord_data['Y-Coord']

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['axes.unicode_minus'] = False
fig, ax = plt.subplots()
ax.plot(x_arr, y_arr, 'o')
ax.set_title('Using hyphen instead of Unicode minus')
plt.show()

In [ ]:
min_x = np.min(x_arr)
min_y = np.min(y_arr)
max_x = np.max(x_arr)
max_y = np.max(y_arr)
h = 100

fake_x = []
fake_y = []

for x in range (min_x, max_x, h):
    for y in range (min_y, max_y, h):
        for idx, row in coord_data.iterrows():
            dist = np.sqrt((x - row['X-Coord'])*(x - row['X-Coord']) + (y - row['Y-Coord'])*(y - row['Y-Coord']))            
            if (dist <= 1000):
                fake_x.append(x)
                fake_y.append(y)
                break
            

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['axes.unicode_minus'] = False
fig, ax = plt.subplots()
ax.plot(x_arr, y_arr, 'o')
ax.plot(fake_x, fake_y, 'x')
ax.set_title('Using hyphen instead of Unicode minus')
plt.show()